# Watermarking

In [5]:
import os
import librosa
import torch
import pandas as pd
import IPython.display as ipd
from tqdm import tqdm
import numpy as np
from pydub import AudioSegment

In [ ]:
clips_path = "../Dataset/cv-corpus-20.0-delta-2024-12-06/en/clips"
watermark_path = "../Dataset/Watermarked Audio"

clips = os.listdir(clips_path)
clips = [i for i in clips if ".mp3" in i[-4:]]

clips_watermarked = os.listdir(watermark_path)
clips_watermarked = [i for i in clips_watermarked if ".mp3" in i[-4:]]
clips_audioseal = [i for i in clips_watermarked if "audioseal" in i]

27408

## AudioSeal

In [3]:
from audioseal import AudioSeal

In [4]:
# Import the model
model = AudioSeal.load_generator("audioseal_wm_16bits")

In [ ]:
# Import audio file
# AudioFile expects a sample rate of 16khz
target_sr = 16000

for clip in tqdm(clips):
    # Load the audio file
    wav, sr = librosa.load(os.path.join(clips_path, clip), sr = target_sr)

    # Play the audio file
    # ipd.Audio(wav, rate=sr)

    # Convert to a PyTorch tensor
    wav_tensor = torch.tensor(wav).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, samples)

    # Generate the watermark
    watermark = model.get_watermark(wav_tensor, sr)
    # Convert PyTorch tensor to NumPy
    watermark = watermark.squeeze().detach().numpy()
    # Add the waterkmark to the audio file
    wav_watermarked = wav + watermark

    # Play the watermarked audio 
    # ipd.Audio(wav_watermarked, rate=sr)

    # Normalize to avoid clipping
    wav_watermarked = np.clip(wav_watermarked, -1.0, 1.0)

    # Convert NumPy array to int16 format for MP3 saving
    wav_int16 = (wav_watermarked * 32767).astype(np.int16)

    # Convert to a pydub AudioSegment
    audio_segment = AudioSegment(
        wav_int16.tobytes(), 
        frame_rate=target_sr, 
        sample_width=2,  # 16-bit PCM
        channels=1  # Mono
    )

    # Define output file name
    output_filename = os.path.join(watermark_path, f"{clip[0:-4]}_watermarked_audioseal.mp3")

    # Export as MP3
    audio_segment.export(output_filename, format="mp3", bitrate="192k")

100%|██████████| 27408/27408 [3:28:52<00:00,  2.19it/s]  


In [13]:
# Import the detector
detector = AudioSeal.load_detector("audioseal_detector_16bits")

In [ ]:
# Test watermarking for imported watermarked audio and original audio

# Load the watermarked audio file
wav_watermarked2, sr = librosa.load(os.path.join(watermark_path, clips_audioseal[0]), sr = target_sr)

# Convert to a PyTorch tensor
wav_watermarked_tensor = torch.tensor(wav_watermarked2).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, samples)

# Detect the watermark
result, _ = detector.detect_watermark(wav_watermarked_tensor, sr)
print(result)

# Detect the watermark
result2, _ = detector.detect_watermark(wav_tensor, sr)
print(result2)

1.0
0.0


## Timbre

## WavMark
[GitHub repo](https://github.com/wavmark/wavmark)

Installation
```bash
pip install wavmark
```

In [11]:
import wavmark
import soundfile
from wavmark.utils import file_reader

In [27]:
clips_path = "../Dataset/cv-corpus-20.0-delta-2024-12-06/en/clips"
watermark_path = "../Dataset/Watermarked Audio"

clips = os.listdir(clips_path)
clips = [i for i in clips if ".mp3" in i[-4:]]

clips_watermarked = os.listdir(watermark_path)
clips_watermarked = [i for i in clips_watermarked if ".mp3" in i[-4:]]
clips_wavmark = [i for i in clips_watermarked if "wavmark" in i]

In [ ]:
# Load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = wavmark.load_model().to(device)

Payload: [0 0 0 1 0 1 1 0 0 1 1 1 1 0 0 0]


In [ ]:
sr = 160000
for clip in tqdm(clips[0:1]):
    # Read host audio
    # the audio should be a single-channel 16kHz wav, you can read it using soundfile:
    # signal, sample_rate = soundfile.read(os.path.join(clips_path, clip))

    # Otherwise, you can use the following function to convert the host audio to single-channel 16kHz format:
    signal = file_reader.read_as_single_channel(os.path.join(clips_path, clip), aim_sr=16000)

    # Create 16-bit payload
    payload = np.random.choice([0, 1], size=16)

    # encode watermark
    watermarked_signal, _ = wavmark.encode_watermark(model, signal, payload, show_progress=True)
    
    # Define output file name
    output_filename = os.path.join(watermark_path, f"{clip[0:-4]}_watermarked_wavmark.wav")

    # you can save it as a new wav:
    soundfile.write(output_filename, watermarked_signal, 16000)

In [24]:
output_filename

'../Dataset/Watermarked Audio/common_voice_en_41585622_watermarked_wavmark.wav'

In [30]:
os.listdir("../Dataset/Watermarked Audio")

['common_voice_en_41669063_watermarked_audioseal.mp3',
 'common_voice_en_41685242_watermarked_audioseal.mp3',
 'common_voice_en_41289642_watermarked_audioseal.mp3',
 'common_voice_en_41798625_watermarked_audioseal.mp3',
 'common_voice_en_41679708_watermarked_audioseal.mp3',
 'common_voice_en_41895923_watermarked_audioseal.mp3',
 'common_voice_en_41774404_watermarked_audioseal.mp3',
 'common_voice_en_41528431_watermarked_audioseal.mp3',
 'common_voice_en_41454299_watermarked_audioseal.mp3',
 'common_voice_en_41886794_watermarked_audioseal.mp3',
 'common_voice_en_41640831_watermarked_audioseal.mp3',
 'common_voice_en_41466605_watermarked_audioseal.mp3',
 'common_voice_en_41366846_watermarked_audioseal.mp3',
 'common_voice_en_41517658_watermarked_audioseal.mp3',
 'common_voice_en_41597043_watermarked_audioseal.mp3',
 'common_voice_en_41746187_watermarked_audioseal.mp3',
 'common_voice_en_41900839_watermarked_audioseal.mp3',
 'common_voice_en_41679617_watermarked_audioseal.mp3',
 'common_v

In [22]:
# Play the watermarked audio 
ipd.Audio(watermarked_signal, rate=sr)